### Use AutoGPT_Project conda environment

In [ ]:
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv(), override = True) 

In [2]:
from langchain import hub 
from langchain_openai import ChatOpenAI 
from langchain.agents import create_react_agent, AgentExecutor 

from langchain_experimental.tools import PythonREPLTool, PythonAstREPLTool 
from langchain_experimental.agents.agent_toolkits import create_csv_agent 

from langchain.tools import Tool

In [3]:
def code_interpreter(query): 
    print("Starting...")
    llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0) 
    
    instructions = """You are an agent designed to write and execute python code to answer questions.
    You have access to a python REPL, which you can use to execute python code.
    If you get an error, debug your code and try again.
    Only use the output of your code to answer the question. 
    You might know the answer without running any code, but you should still run the code to get the answer.
    If it does not seem like you can write code to answer the question, just return "I don't know" as the answer."""
    
    base_prompt = hub.pull("langchain-ai/react-agent-template") 
    
    prompt = base_prompt.partial(instructions = instructions) 
    
    tools = [PythonREPLTool(), PythonAstREPLTool()] 
    
    agent = create_react_agent(llm = llm, prompt = prompt, tools = tools) 
    
    python_agent_executor = AgentExecutor(
        agent = agent, 
        tools = tools, 
        verbose = True, 
        handle_parsing_errors = True
    )
    
    csv_agent_executor: AgentExecutor = create_csv_agent(
        llm = llm, 
        path = "./episode_info.csv", 
        verbose = True, 
        allow_dangerous_code = True, 
        )
    
    def python_agent_executor_wrapper(original_prompt: str) -> dict[str, any]: 
        return python_agent_executor.invoke(input = {"input": original_prompt})
    
    tools_for_router = [
        Tool(
            name = "Python Agent", 
            func = python_agent_executor_wrapper, 
            description = """Useful when you need to transform natural language to python and execute python code, 
                           returning thr results of the code execution,
                           DO NOT ACCEPT CODE AS INPUT""", 
        ), 
        Tool( 
             name = "CSV Agent", 
             func = csv_agent_executor.invoke, 
             description = """Useful when you need to answer questions over episode_info.csv, 
                           takes as input the entire question and returns the answer after running pandas calculations""")
    ]
    
    router_prompt = base_prompt.partial(instructions = "Output only natural language, not code") 
    
    router_agent = create_react_agent(
        prompt = router_prompt, 
        llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0), 
        tools = tools_for_router
    )
    
    router_agent_executor = AgentExecutor(
        agent = router_agent, 
        tools = tools_for_router, 
        verbose = True, 
        handle_parsing_errors = True
    )
    
    router_response = router_agent_executor.invoke(input = {"input": query}) 
    
    return router_response

In [ ]:
output_1 = code_interpreter(query = "Which season has the most episodes?") 

print(output_1)

In [ ]:
output_2  = code_interpreter(query = "Write and execute python code to print 'Hello World'") 

print(output_2["output"])